In [29]:
!pip install neo4j
import requests
import json
from neo4j import GraphDatabase
import time

In [30]:
# в данном блоке я указываю токен с кинопоиска, с поможью которого можно делать запросы API на кинопоиске

api_token = 'c197339c-b5d3-4ffa-92a3-4beaf044cd59'

#  здесь указываются сайт с запросами и запросы, которые мне понадобятся для получения данных
url_api = 'https://kinopoiskapiunofficial.tech'
url_films = '/api/v2.2/films'
url_staff = '/api/v1/staff'

In [31]:
# на данном этапе я записываю данные от Neo4j для того, чтобы работать с бд Neo4j

URI = "neo4j+s://42bd20e7.databases.neo4j.io"
AUTH = ("neo4j", "yKQw4OtYD4Bvuj-Te7cUqhb0ZoM0L2YrXIPPdutWg8U")

In [32]:
# на данном этапе выполняется функция, которая выполняет запрос по указанному выше URL

def get_request(url_suffix, params=None):
    headers = {
        'X-API-KEY': api_token,
        'Content-Type': 'application/json',
    }

    # Выполнение GET-запроса
    response = requests.get(url_api + url_suffix, headers=headers, params=params)

    return response

In [33]:
# на данном этапе для дальнейшей удобной работы, полученные данные в тексовой форме, переделаем в json
response = get_request(url_films)
json_data = json.loads(response.text)
json_data

{'total': 100,
 'totalPages': 5,
 'items': [{'kinopoiskId': 1252447,
   'imdbId': 'tt9257638',
   'nameRu': 'Лорды раздевалки',
   'nameEn': None,
   'nameOriginal': 'Lords of the Lockerroom',
   'countries': [{'country': 'США'}],
   'genres': [{'genre': 'спорт'}, {'genre': 'для взрослых'}],
   'ratingKinopoisk': 9.4,
   'ratingImdb': 9.3,
   'year': 1999,
   'type': 'VIDEO',
   'posterUrl': 'https://kinopoiskapiunofficial.tech/images/posters/kp/1252447.jpg',
   'posterUrlPreview': 'https://kinopoiskapiunofficial.tech/images/posters/kp_small/1252447.jpg'},
  {'kinopoiskId': 5260016,
   'imdbId': None,
   'nameRu': 'Попкульт',
   'nameEn': None,
   'nameOriginal': None,
   'countries': [{'country': 'Россия'}],
   'genres': [{'genre': 'документальный'}],
   'ratingKinopoisk': 9.4,
   'ratingImdb': None,
   'year': 2022,
   'type': 'TV_SERIES',
   'posterUrl': 'https://kinopoiskapiunofficial.tech/images/posters/kp/5260016.jpg',
   'posterUrlPreview': 'https://kinopoiskapiunofficial.tech/i

In [34]:
#из-за того, что данные отображаются не очень корректно, страны и жанры находятся будто в массиве, сделаю обработку данных
movies_data_list = []
for movie in json_data['items']:
    countries_arr = []
    genres_arr = []
    for country in movie['countries']:
        countries_arr.append(country['country'])
    for genre in movie['genres']:
        genres_arr.append(genre['genre'])

    movie['countries'] = countries_arr
    movie['genres']= genres_arr
    movies_data_list.append(movie)

In [35]:
movies_data_list

[{'kinopoiskId': 1252447,
  'imdbId': 'tt9257638',
  'nameRu': 'Лорды раздевалки',
  'nameEn': None,
  'nameOriginal': 'Lords of the Lockerroom',
  'countries': ['США'],
  'genres': ['спорт', 'для взрослых'],
  'ratingKinopoisk': 9.4,
  'ratingImdb': 9.3,
  'year': 1999,
  'type': 'VIDEO',
  'posterUrl': 'https://kinopoiskapiunofficial.tech/images/posters/kp/1252447.jpg',
  'posterUrlPreview': 'https://kinopoiskapiunofficial.tech/images/posters/kp_small/1252447.jpg'},
 {'kinopoiskId': 5260016,
  'imdbId': None,
  'nameRu': 'Попкульт',
  'nameEn': None,
  'nameOriginal': None,
  'countries': ['Россия'],
  'genres': ['документальный'],
  'ratingKinopoisk': 9.4,
  'ratingImdb': None,
  'year': 2022,
  'type': 'TV_SERIES',
  'posterUrl': 'https://kinopoiskapiunofficial.tech/images/posters/kp/5260016.jpg',
  'posterUrlPreview': 'https://kinopoiskapiunofficial.tech/images/posters/kp_small/5260016.jpg'},
 {'kinopoiskId': 1201206,
  'imdbId': None,
  'nameRu': None,
  'nameEn': None,
  'nameOr

In [36]:
# теперь начнем формировать БД в Neo4j
# функция для выполнения запросов к базе данных Neo4j
def run_query(query, parameters=None):
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        with driver.session() as session:
            result = session.run(query, parameters)
            return result

In [37]:
# создаем узлы для фильмов
def create_movies(data_list):
    query = (
        "UNWIND $movies AS movie "
        "CREATE (:Movie {"
            "ratingKinopoisk: movie.ratingKinopoisk, "
            "nameOriginal: movie.nameOriginal, "
            "genres: movie.genres, "
            "year: movie.year, "
            "type: movie.type, "
            "countries: movie.countries, "
            "kinopoiskId: movie.kinopoiskId, "
            "imdbId: movie.imdbId, "
            "ratingImdb: movie.ratingImdb"

        "})"
    )
    parameters = {"movies": data_list}
    run_query(query, parameters)

In [38]:
create_movies(movies_data_list)

In [39]:
# то же самое проделываем для получения данных о персонале(режисерах,актерах и тд)
staff_data_list = []
for movie in movies_data_list:
    response = get_request(url_staff, {'filmId': movie['kinopoiskId']})

# для того, чтобы объеденить между собой фильмы, жанры и персонал добавляем id
    for staff in json.loads(response.text):
        if 'filmsId' in staff.keys():
            staff['filmsId'].append(movie['kinopoiskId'])
        else:
            staff['filmsId'] = [movie['kinopoiskId']]

        staff_data_list.append(staff)

    time.sleep(0.2) # (так как у API есть ограничение "Каждый пользователь имеет ограничение в 20 req/sec")

In [40]:
staff_data_list

[{'staffId': 3978602,
  'nameRu': 'Мэттью Мур',
  'nameEn': 'Matthew Moore',
  'description': None,
  'posterUrl': 'https://kinopoiskapiunofficial.tech/images/actor_posters/kp/3978602.jpg',
  'professionText': 'Режиссеры',
  'professionKey': 'DIRECTOR',
  'filmsId': [1252447]},
 {'staffId': 1245312,
  'nameRu': 'Ник Стил',
  'nameEn': 'Nick Steel',
  'description': 'Wrestler (в титрах: Nick Steel)',
  'posterUrl': 'https://kinopoiskapiunofficial.tech/images/actor_posters/kp/1245312.jpg',
  'professionText': 'Актеры',
  'professionKey': 'ACTOR',
  'filmsId': [1252447]},
 {'staffId': 3981260,
  'nameRu': 'Ван Даркхолм',
  'nameEn': 'Van Darkholme',
  'description': 'Leatherman',
  'posterUrl': 'https://kinopoiskapiunofficial.tech/images/actor_posters/kp/3981260.jpg',
  'professionText': 'Актеры',
  'professionKey': 'ACTOR',
  'filmsId': [1252447]},
 {'staffId': 3996077,
  'nameRu': 'Билли Херрингтон',
  'nameEn': 'Billy Herrington',
  'description': 'Wrestler',
  'posterUrl': 'https://ki

In [41]:
# создаем узлы для персонала
def create_staff(data_list):
    query = (
        "UNWIND $staffs AS staff "
        "CREATE (:Staff {"
            "staffId: staff.staffId, "
            "nameRu: staff.nameRu, "
            "nameEn: staff.nameEn, "
            "description: staff.description, "
            "professionText: staff.professionText, "
            "filmsId: staff.filmsId "
        "})"
    )
    parameters = {"staffs": data_list}
    run_query(query, parameters)

In [42]:
create_staff(staff_data_list)

In [43]:
# по такому же принципу формаруем данные по жанрам
unique_list = []
genres_data_list = []
for movie in movies_data_list:
    for genre in movie['genres']:
        genres_data_list.append({'name': genre, 'filmsId': movie['kinopoiskId']})
print(genres_data_list)

[{'name': 'спорт', 'filmsId': 1252447}, {'name': 'для взрослых', 'filmsId': 1252447}, {'name': 'документальный', 'filmsId': 5260016}, {'name': 'музыка', 'filmsId': 1201206}, {'name': 'короткометражка', 'filmsId': 1201206}, {'name': 'мультфильм', 'filmsId': 962472}, {'name': 'детский', 'filmsId': 962472}, {'name': 'документальный', 'filmsId': 1309325}, {'name': 'фантастика', 'filmsId': 1072974}, {'name': 'мультфильм', 'filmsId': 1072974}, {'name': 'детский', 'filmsId': 1072974}, {'name': 'мультфильм', 'filmsId': 952236}, {'name': 'детский', 'filmsId': 952236}, {'name': 'комедия', 'filmsId': 45319}, {'name': 'мультфильм', 'filmsId': 45319}, {'name': 'семейный', 'filmsId': 45319}, {'name': 'короткометражка', 'filmsId': 45319}, {'name': 'фэнтези', 'filmsId': 300980}, {'name': 'комедия', 'filmsId': 300980}, {'name': 'мюзикл', 'filmsId': 300980}, {'name': 'короткометражка', 'filmsId': 300980}, {'name': 'документальный', 'filmsId': 674243}, {'name': 'боевик', 'filmsId': 1339977}, {'name': 'ко

In [44]:
# чтобы жанры у фильмов не повторялись - создадим отдельный словарь с неповторяющимися словами
unique_names = {}

for item in genres_data_list:
    name = item['name']
    films_id = item['filmsId']

    if name in unique_names:
        unique_names[name].append(films_id)
    else:
        unique_names[name] = [films_id]

# Преобразуем словарь в массив
genres_data_list = [{'name': name, 'filmsId': films_ids} for name, films_ids in unique_names.items()]

In [45]:
print(genres_data_list)

[{'name': 'спорт', 'filmsId': [1252447]}, {'name': 'для взрослых', 'filmsId': [1252447]}, {'name': 'документальный', 'filmsId': [5260016, 1309325, 674243, 351771, 1007472, 2500772]}, {'name': 'музыка', 'filmsId': [1201206, 351771]}, {'name': 'короткометражка', 'filmsId': [1201206, 45319, 300980, 1339977, 1043658]}, {'name': 'мультфильм', 'filmsId': [962472, 1072974, 952236, 45319, 1368895]}, {'name': 'детский', 'filmsId': [962472, 1072974, 952236]}, {'name': 'фантастика', 'filmsId': [1072974, 1368895]}, {'name': 'комедия', 'filmsId': [45319, 300980, 1339977, 77044, 1043658, 972889]}, {'name': 'семейный', 'filmsId': [45319]}, {'name': 'фэнтези', 'filmsId': [300980]}, {'name': 'мюзикл', 'filmsId': [300980, 1003587]}, {'name': 'боевик', 'filmsId': [1339977]}, {'name': 'мелодрама', 'filmsId': [77044]}, {'name': 'концерт', 'filmsId': [351771]}, {'name': 'биография', 'filmsId': [1003587]}, {'name': 'реальное ТВ', 'filmsId': [1399205, 972889]}, {'name': 'военный', 'filmsId': [1368895]}]


In [46]:
# на данном этапе происходит создание узлов с жанрами фильмов

def create_genres(data_list):
    query = (
        "UNWIND $genres AS genre "
        "CREATE (:Genres { name: genre.name, filmsId: genre.filmsId })"
    )
    parameters = {"genres": data_list}
    run_query(query, parameters)

In [47]:
create_genres(genres_data_list)

In [55]:
# на данном этапе мы создаем связи между жанром и фильмом

def relationship_movies_genres():
  query = (
    "MATCH (g:Genres), (m:Movie) "
        "WHERE m.kinopoiskId IN g.filmsId "
        "MERGE  (g)-[:Жанры]->(m) "
)
  run_query(query)

In [56]:
relationship_movies_genres()

In [57]:
# на данном этапе мы создаем связи между фильмом и актерами(персоналом)

def relationship_movies_staff():
    query = (
        "MATCH (s:Staff), (m:Movie) "
        "WHERE m.kinopoiskId IN s.filmsId "
        "MERGE (m)-[:Актеры]->(s) "
    )
    run_query(query)

In [58]:
relationship_movies_staff()